Inspired by https://f1metrics.wordpress.com/2014/10/03/building-a-race-simulator/



In [1]:
import fastf1 as ff1
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import matplotlib.patches as mpatches
from race import *
from constants import *
from practice import *
from scipy import stats

# Enable the cache by providing the name of the cache folder
ff1.Cache.enable_cache('cache')

#### FP2

In [2]:
FP2 = ff1.get_session(2022, 'Hungary', 'FP2')
FP2.load();

core           INFO 	Loading data for Hungarian Grand Prix - Practice 2 [v2.2.9]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '55', '47', '44', '31', '24', '23', '22', '20', '18', '16', '14', '11', '10', '6', '5', '4', '3', '63', '77']


In [3]:
# plot_dry_tyre_models_all_drivers(FP2.laps, get_all_driver_names(FP2.laps))

#### Data Wrangling - Getting the Race Pace of each driver from FP2 Long Run Simulation

In [9]:
# Create a new dataframe for storing drivers' racepace
fp2_race_sim = pd.DataFrame(columns = ['Driver', 'MeanLapTime', 'Compound', 'NoOfLaps', 'LapTimes', 'LapNumbers'])
fp2_race_sim

,Driver,MeanLapTime,Compound,NoOfLaps,LapTimes,LapNumbers


In [10]:
for driver in get_all_driver_names(FP2.laps):
    long_pace_tuple = extract_long_run_pace_from_longest_practice_stint(FP2.laps, driver)
    
    # Create a new entry to the racepace dataframe by creating a row, which will be appended to the dataframe (fp2_race_sim) in the next line.
    # Note the order of the items in the list.
    new_row = [driver,  
               sum(long_pace_tuple[0]) /  len(long_pace_tuple[0]),
               long_pace_tuple[1], len(long_pace_tuple[0]), 
               long_pace_tuple[0],
               long_pace_tuple[2]]
    # Append the new entry.
    fp2_race_sim.loc[len(fp2_race_sim)] = new_row
    
fp2_race_sim = fp2_race_sim.sort_values('MeanLapTime')

# TODO: For the drivers with NoOfLaps less than 3, we can "interpolate?extrapolate?guess?" their laptime by getting the RANGE of the laptime
# of the remaining drivers and assign a laptime based on their TEAM, with teams with faster cars getting a better laptime. For example, if the team
# is the 6th best team, an example of the guesswork value can be:
# laptime = min + (max - min) * 6 / NO_OF_TEAMS
# Note that this is assuming that the time delta between each team is constant. This is a simple solution for now, but not necessarily fair.

# TODO: Consider the different tyre compounds used

In [11]:
for driver in get_all_driver_names(FP2.laps):
    df = FP2.laps.pick_driver(driver).pick_accurate()
    fp2_race_sim.loc[fp2_race_sim['Driver'] == driver, 'SpeedFL'] = max(df['SpeedFL'])
    fp2_race_sim.loc[fp2_race_sim['Driver'] == driver, 'SpeedST'] = max(df['SpeedST'])

In [14]:
fp2_race_sim = fp2_race_sim.reset_index(drop=True)

# Rearrange the columns
fp2_race_sim = fp2_race_sim[['Driver', 'MeanLapTime', 'SpeedFL', 'SpeedST', 'Compound', 'NoOfLaps', 'LapTimes', 'LapNumbers']]
fp2_race_sim

,Driver,MeanLapTime,SpeedFL,SpeedST,Compound,NoOfLaps,LapTimes,LapNumbers
0,HAM,80.853000,253.0,302.0,MEDIUM,1,[80.85300000000001],[2.0]
1,SAI,83.624000,257.0,NaN,MEDIUM,7,"[83.185, 83.22500000000001, 83.44000000000001,...","[12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0]"
2,LEC,83.698286,253.0,303.0,MEDIUM,7,"[83.57400000000001, 83.552, 83.435, 83.4, 83.7...","[10.0, 11.0, 12.0, 13.0, 15.0, 16.0, 17.0]"
3,BOT,83.834000,255.0,305.0,MEDIUM,1,[83.834],[14.0]
4,NOR,83.917429,253.0,306.0,SOFT,7,"[83.25, 83.333, 84.006, 83.667, 84.56700000000...","[7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 14.0]"
5,OCO,84.055000,253.0,304.0,SOFT,1,[84.055],[10.0]
6,VER,84.173000,255.0,305.0,MEDIUM,8,"[83.771, 83.941, 84.012, 84.18100000000001, 84...","[12.0, 13.0, 14.0, 15.0, 16.0, 18.0, 20.0, 22.0]"
7,RIC,84.187333,255.0,304.0,SOFT,6,"[84.21600000000001, 83.796, 84.15400000000001,...","[6.0, 7.0, 8.0, 9.0, 12.0, 13.0]"
8,VET,84.585000,255.0,305.0,MEDIUM,8,"[84.03500000000001, 84.138, 84.71000000000001,...","[11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0]"
9,PER,84.781500,254.0,306.0,MEDIUM,10,"[84.558, 84.578, 84.39, 84.577, 84.727, 84.972...","[11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 19...."


In [15]:
# Share variables between jupyter notebooks.
%store fp2_race_sim

Stored 'fp2_race_sim' (DataFrame)
